In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
import tokamakTK
from tokamakTK import get_ECT_regression

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from collections import Counter
from scipy.stats import f
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

pd.set_option('display.max_columns', None)
path = "../data/"
fig_path = "../../../LATEX/Latex Images/"

plt.rc('font',family = 'serif')

features = ['IP', 'BT', 'NEL', 'PLTH', 'RGEO', 'KAREA', 'EPS', 'MEFF']

In [9]:
# Obtained from Optimization

min_subset_ids_6357 = pd.read_csv(path+"R_ids_alpha_0.6357.csv")
min_subset_ids_9998 = pd.read_csv(path+"R_ids_alpha_0.9998.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# Removing Spherical TOKAMAKS
DB5 = DB5[~DB5["TOK"].isin(['START','MAST','NSTX'])]

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label_6357",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_6357.id)].index), "label_6357"] = 1

DB5.insert(loc=2,column="label_9998",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_9998.id)].index), "label_9998"] = 1

DB5.insert(0, 'intercept', np.ones(len(DB5)))


print(
    "  Subset that decrease alpha-R to 0.6357\n--------\n" +
    f"{ round( (len(min_subset_ids_6357)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_6357)/len(DB5))*100 ,2)  }% did not affect alpha_R" +
    "\n\n\n  Subset that decrease alpha-R to 0.9998\n--------\n" +
    f"{ round( (len(min_subset_ids_9998)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_9998)/len(DB5))*100 ,2)  }% did not affect alpha_R"
)

  Subset that decrease alpha-R to 0.6357
--------
24.37% affected alpha_R
75.63% did not affect alpha_R


  Subset that decrease alpha-R to 0.9998
--------
10.27% affected alpha_R
89.73% did not affect alpha_R


In [10]:
all_cases = [DB2.copy(),
             DB5.copy(),
             DB5[DB5.label_6357.isin([1]) | DB5.id.isin(DB2.id.values)],
             DB5[DB5.label_6357.isin([0])],
             DB5[DB5.label_9998.isin([1]) | DB5.id.isin(DB2.id.values)],
             DB5[DB5.label_9998.isin([0])]
]

def get_FPvalues(full_model, reduced_model):
    return (anova_lm(reduced_model, full_model)["F"][1], 
            anova_lm(reduced_model, full_model)["Pr(>F)"][1])

In [11]:
# Creating table with F-test and P-test values per case
TESTS = pd.DataFrame(np.zeros((len(features), len(all_cases)*2)), 
                    columns=['FV0', 'PV0', 'FV1', 'PV1', 'FV2', 'PV2', 'FV3', 'PV3', 'FV4', 'PV4', 'FV5', 'PV5'],
                    index=features)

In [12]:
for i, data_ in enumerate(all_cases):
    for f in features:
        
        reduced_features = features.copy()
        reduced_features.remove(f)
        
        # get_ECT_regression takes the regression in log-space with intercept
        
        full_regression = get_ECT_regression(data_)
        reduced_regression = get_ECT_regression(data_, features=reduced_features)
        
        TESTS.loc[f, f"FV{i}"] = np.round(get_FPvalues(full_regression, reduced_regression)[0],5)
        TESTS.loc[f, f"PV{i}"] = np.round(get_FPvalues(full_regression, reduced_regression)[1],5)
        
TESTS.index.rename("removed feature", inplace=True)
c = ["F_db2", "P_db2", "F_db5", "P_db5", "F_d0.6357", "P_d0.6357", # (d): decreasing; (u): unaffected
                                         "F_u0.6357", "P_u0.6357", "F_d0.9998", "P_d0.9998",
                                                                   "F_u0.9998", "P_u0.9998"]
TESTS.columns = c

In [13]:
TESTS

,F_db2,P_db2,F_db5,P_db5,F_d0.6357,P_d0.6357,F_u0.6357,P_u0.6357,F_d0.9998,P_d0.9998,F_u0.9998,P_u0.9998
removed feature,,,,,,,,,,,,
IP,961.58446,0.0,5181.43523,0.00000,4152.28514,0.00000,2270.72499,0.0,2047.64264,0.00000,3958.40000,0.00000
BT,138.99369,0.0,4.79280,0.02862,0.10283,0.74848,167.09337,0.0,25.81100,0.00000,3.12710,0.07706
NEL,494.27457,0.0,231.23438,0.00000,99.34833,0.00000,1770.86448,0.0,1.32319,0.25017,596.25357,0.00000
PLTH,2674.96819,0.0,9305.98576,0.00000,2038.38644,0.00000,13804.40093,0.0,1634.31731,0.00000,10291.97609,0.00000
RGEO,2022.42722,0.0,2092.03540,0.00000,236.78331,0.00000,5458.33241,0.0,457.26387,0.00000,2987.74270,0.00000
KAREA,94.66733,0.0,13.33678,0.00026,0.76233,0.38268,240.41914,0.0,12.94685,0.00033,44.85694,0.00000
EPS,131.13720,0.0,0.13161,0.71679,145.32482,0.00000,375.20589,0.0,13.62917,0.00023,37.33309,0.00000
MEFF,36.50700,0.0,192.29798,0.00000,27.40806,0.00000,188.62652,0.0,15.54385,0.00008,210.30714,0.00000
